In [65]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [66]:
df = pd.read_csv('T10Y2Y.csv')

In [67]:
print(df.shape)
df.head()

(11437, 2)


,DATE,T10Y2Y
0,1976-06-01,0.68
1,1976-06-02,0.71
2,1976-06-03,0.70
3,1976-06-04,0.77
4,1976-06-07,0.79


In [68]:
df.dtypes

DATE      object
T10Y2Y    object
dtype: object

In [69]:
df = df[df['T10Y2Y'] != '.']

In [70]:
def convert_yield(row):
    if row != '.':
        return float(row)
    else: 
        return 0.00

In [71]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['T10Y2Y'] = df['T10Y2Y'].map(lambda x: float(x))

In [76]:
daily_yield = df.groupby(['DATE'])['T10Y2Y'].sum()
daily_yield = pd.DataFrame(daily_yield, columns=['T10Y2Y'])

In [77]:
daily_yield

,T10Y2Y
DATE,
1976-06-01,0.68
1976-06-02,0.71
1976-06-03,0.70
1976-06-04,0.77
1976-06-07,0.79
...,...
2020-03-26,0.53
2020-03-27,0.47
2020-03-30,0.47


In [93]:
f = go.Figure()
f.add_trace(go.Scatter(x=daily_yield.index, 
                       y=daily_yield.T10Y2Y, 
                       line_color='rgb(0, 0, 100)', 
                       ))

f.update_layout(title_text=
                '10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity')
f.add_shape(
        # Line Horizontal
            type="line",
            x0=start_dt,
            y0=0,
            x1=end_dt,
            y1=0,
            line=dict(
                color="LightSeaGreen",
                width=4,
                dash="dashdot",
            ))

f.show()

NameError: name 'start_dt' is not defined